In [ ]:
import pandas as pd

In [ ]:
import numpy as np

On commence par charger les données qui sont dans un fichier .csv avec comme séparateur une ,

In [ ]:
file_name="dispersion_10240.csv"

In [ ]:
xy_train=pd.read_csv(file_name,header=0)

Visualisation

In [ ]:
print(xy_train)

Pour voir le format des données

In [ ]:
xy_train.shape

In [ ]:
xy_train['model'] = xy_train['model'].astype('category')

In [ ]:
xy_train['ihaze'] = xy_train['ihaze'].astype('category')

In [ ]:
xy_train['clouds'] = xy_train['clouds'].astype('category')

In [ ]:
x_tot=np.array(xy_train[['cap','Mach','pitch','E_e_ai','power_setting','altitude','roll','yaw','hour','model','ihaze','hbase','hr','ta','vis','clouds']])

In [ ]:
y_tot=np.array(xy_train['IRS'])

In [ ]:
x_tot.shape

**Modèle de type régression linéaire**

In [ ]:
import sklearn.linear_model as lm
linreg = lm.LinearRegression()

In [ ]:
y_tot.shape

Il faut un vecteur de type (10240,1) pour l'apprentissage, on fait un reshape

In [ ]:
y_tot=y_tot.reshape(10240,1)

In [ ]:
lr=linreg.fit(x_tot, y_tot)

In [ ]:
lr.coef_

Si on veut normaliser les données

In [ ]:
from sklearn import preprocessing

In [ ]:
x_scaled = preprocessing.scale(x_tot)

Régression linéaire avec input normalisés

In [ ]:
lr=linreg.fit(x_scaled, y_tot)

In [ ]:
lr.coef_

Tracés

In [ ]:
%pylab inline

In [ ]:
plt.plot(lr.predict(x_scaled), y_tot,'o')

**Modèle de type MLP**

In [ ]:
y_tot=log(y_tot)

On sépare les données entre apprentissage et test (60%)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_tot, test_size=0.6)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
nnet=MLPRegressor(hidden_layer_sizes=(11,),max_iter=5000,activation='tanh',alpha=0.05,solver='lbfgs')

In [ ]:
y_train.shape

In [ ]:
y_train=y_train.reshape(4096,)

In [ ]:
y_test.shape

In [ ]:
y_test=y_test.reshape(6144,)

In [ ]:
nnet.fit(x_train,y_train)

In [ ]:
predictions = nnet.predict(x_test)
print(max(predictions))

In [ ]:
plt.plot(predictions, y_test,'o')

In [ ]:
nnet.score(x_test,y_test)

**Exercice 1** : Tester avec d'autres paramètres

**Exercice 2** : Tester d'autres métamodèles

In [ ]:
file_name="basesvm_avions.csv"

In [ ]:
xy_classif=pd.read_csv(file_name,delimiter=';', header=None)

In [ ]:
xy_classif.describe()

In [ ]:
xy_classif.shape

In [ ]:
xy_classif.head()

In [ ]:
from sklearn import svm, metrics

In [ ]:
x_tot=np.array(xy_classif[np.arange(225)])

In [ ]:
x_tot.shape

In [ ]:
y_tot=np.array(xy_classif[225])

In [ ]:
x0_train=x_tot[0:500,:]
x1_train=x_tot[10000:10500,:]
x2_train=x_tot[20000:20500,:]
x_train=np.concatenate((x0_train,x1_train,x2_train),axis=0)

In [ ]:
y0_train=y_tot[0:500]
y1_train=y_tot[10000:10500]
y2_train=y_tot[20000:20500]
y_train=np.concatenate((y0_train,y1_train,y2_train),axis=0)

In [ ]:
x0_test=x_tot[501:1500,:]
x1_test=x_tot[10501:11500,:]
x2_test=x_tot[20501:21500,:]
x_test=np.concatenate((x0_test,x1_test,x2_test),axis=0)

In [ ]:
y0_test=y_tot[501:1500]
y1_test=y_tot[10501:11500]
y2_test=y_tot[20501:21500]
y_test=np.concatenate((y0_test,y1_test,y2_test),axis=0)

In [ ]:
y_train.shape

In [ ]:
svm_iris=svm.SVC(kernel='rbf',C=10)

In [ ]:
svm_iris.fit(x_train,y_train)

In [ ]:
y_pred=svm_iris.predict(x_test)

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n%s" % cm)